In [1]:
import os

In [16]:
os.chdir("../")

In [17]:
# make sure to present in the root directory of the project.
pwd

'E:\\Projects\\MLOPS\\kidney-disease-classification-mlops'

In [10]:
# creating training config entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    

In [19]:
from kidneyDiseaseClassifier import logger
from kidneyDiseaseClassifier.constants import *
from kidneyDiseaseClassifier.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(self, model_name= "VGG16",  config_filepath= CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.model_name = model_name.upper()
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)[self.model_name]       # loading params for the specific model
        create_directories([self.config.artifacts_root])
    
    def get_training_config(self)->TrainingConfig:
        training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        training_data = os.path.join(self.config.data_ingestion_kaggle.local_data_file, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([Path(training.root_dir)])
        
        logger.info(f"training data path: {training_data}")
        logger.info(f"{os.listdir(training_data)}")
        
        training_config= TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )
        
        return training_config

In [26]:
import os
import time
import tensorflow as tf
import urllib.request as request
from zipfile import ZipFile

In [27]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.model = None
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            )
        
    def train_valid_generator(self):
        datagenerator_kwars = dict(
            rescale=1./255,
            validation_split=0.2
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwars)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwars
            )
        else:   
            train_datagenerator = valid_datagenerator
            
        self.train_genrator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)
        
        
    def train(self, callbacks_list:list):
        self.steps_per_epoch = self.train_genrator.samples // self.train_genrator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        
        
        self.model.fit(
            self.train_genrator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callbacks_list
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [30]:
%%timeit
try:
    config = ConfigurationManager(model_name="VGG16")
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callbacks_list=None)
except Exception as e:
    raise e
    

[2023-12-11 00:45:50,974]: INFO: common: yaml file: config\config.yaml loaded successfully!
[2023-12-11 00:45:50,979]: INFO: common: yaml file: params.yaml loaded successfully!
[2023-12-11 00:45:50,981]: INFO: common: Created directory at : artifacts
[2023-12-11 00:45:50,982]: INFO: common: Created directory at : artifacts\training
[2023-12-11 00:45:50,983]: INFO: 3552435675: training data path: artifacts/data_ingestion/kaggle/data\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone
[2023-12-11 00:45:50,985]: INFO: 3552435675: ['Cyst', 'Normal', 'Stone', 'Tumor']
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
622/622 [==============================] - 2345s 4s/step - loss: 1.0957 - accuracy: 0.5497 - val_loss: 9.4967 - val_accuracy: 0.2988


E:\Projects\MLOPS\kidney-disease-classification-mlops\venv\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-12-11 01:24:59,495]: INFO: common: yaml file: config\config.yaml loaded successfully!
[2023-12-11 01:24:59,499]: INFO: common: yaml file: params.yaml loaded successfully!
[2023-12-11 01:24:59,500]: INFO: common: Created directory at : artifacts
[2023-12-11 01:24:59,503]: INFO: common: Created directory at : artifacts\training
[2023-12-11 01:24:59,504]: INFO: 3552435675: training data path: artifacts/data_ingestion/kaggle/data\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone
[2023-12-11 01:24:59,505]: INFO: 3552435675: ['Cyst', 'Normal', 'Stone', 'Tumor']
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
 10/622 [..............................] - ETA: 32:04 - loss: 1.5487 - accuracy: 0.2937

KeyboardInterrupt: 